In [1]:
"""
Copyright (C) 2022 Dominik Lentrodt

This file is part of pyrot.

pyrot is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

pyrot is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with pyrot.  If not, see <http://www.gnu.org/licenses/>.
"""

%matplotlib inline

### Import packages

In [2]:
# for calculations
import numpy as np
import scipy as sp
from scipy.signal import argrelextrema

# for plotting
import matplotlib
import matplotlib.pylab as plt

In [3]:
### for development ###
# import os
# import sys
# module_path = os.path.abspath(os.path.join('../src'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

# import pyrot

In [4]:
### if using installed distribution ###
import pyrot

ModuleNotFoundError: No module named 'pyrot'

### Example system calculation

Define system

In [ ]:
tThin = 0.01 #thin layer
N = [1.0,    4.0, 1.0,  4.0, 1.0] # Refractive index of each layer
T = [ -1, tThin, 1.0, tThin,  -1] # Thicknesses of each layer

omega = np.linspace(0.001, 15., 200000) # incident frequency / wavenumber

### atom parameters ###
atom_pos   =   0.51 # Position of the atom (measured relative to left cavity surface)
atom_dpol  =   0.05 # dipole moment
atom_om    =   4.35 # transition energy
atom_gamma =   0.00  # spontaneous decay rate into non-radiative channels

atoms_params = [[atom_pos, atom_dpol, atom_om, atom_gamma],]

In [ ]:
cavity = pyrot.Cavity1d(N, T)
atom   = pyrot.CavityAtoms1d(N, T, atoms_params)

Calculate **empty cavity** scattering

In [ ]:
%%time
R_empty = cavity.reflection_intensity(omega)
T_empty = cavity.transmission_intensity(omega)

Calculate reflection minimum

In [ ]:
max_inds = argrelextrema(R_empty, np.less)
ref_min = omega[max_inds[0][1]]+0.000001 # small number to avoid divide by zeroes
print(ref_min)

Calculate cavity scattering **with atom**

In [ ]:
%%time
R_atom = atom.linear_reflection_intensity(omega)
T_atom = atom.linear_transmission_intensity(omega)

Print linear refractive index with atom delta approximated as thin layer

In [ ]:
n_int, t_int = atom.linear_layer_system_with_atom(omega)
print(n_int)
print(t_int)

### Plot

In [ ]:
t_tot = np.sum(T[1:-1])
outside_buffer = 0.2
depth = np.linspace(-outside_buffer, t_tot+outside_buffer, 10000)

#cavity.draw_cav(depth)
atom.draw_cav(depth)

In [ ]:
### reflection ###
plt.figure(figsize=(6*2,4))

plt.subplot(121, xlabel='Incident frequency [arb. units]', ylabel='Reflectance',
            title='Reflectance of example cavity')
plt.plot(omega, R_atom, label='with atom')
plt.plot(omega, R_empty, '--', label='empty cavity')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')
plt.legend(loc=1, fontsize=8)

plt.subplot(122, xlabel='Incident frequency [arb. units]', ylabel='Reflectance',
            title='Reflectance of example cavity (zoom around atom frequency)')
plt.plot(omega, R_atom, label='with atom')
plt.plot(omega, R_empty, '--', label='empty cavity')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')

zoom_dif = 0.05
plt.xlim([atom_om-zoom_dif, atom_om+zoom_dif])
plt.legend(loc=1, fontsize=8)

plt.show()

### transmission ###
plt.figure(figsize=(6*2,4))

plt.subplot(121, xlabel='Incident frequency [arb. units]', ylabel='Transmittance',
            title='Transmittance of cavity')
plt.plot(omega, T_atom, label='with atom')
plt.plot(omega, T_empty, '--', label='empty cavity')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')
plt.legend(loc=4, fontsize=8)

plt.subplot(122, xlabel='Incident frequency [arb. units]', ylabel='Transmittance',
            title='Transmittance of cavity (zoom around atom frequency)')
plt.plot(omega, T_atom, label='with atom')
plt.plot(omega, T_empty, '--', label='empty cavity')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')

zoom_dif = 0.05
plt.xlim([atom_om-zoom_dif, atom_om+zoom_dif])
plt.legend(loc=4, fontsize=8)

plt.show()